In [ ]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
 
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline

#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

#preprocess.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#dl libraraies
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.utils import to_categorical

# specifically for cnn
from tensorflow.keras.layers import Dropout, Flatten,Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
 
import tensorflow as tf
import random as rn

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image

In [ ]:
#Tensorboard Things

%load_ext tensorboard
import tensorflow as tf
import datetime, os


In [ ]:
from dataset_division import *
from dataset_aug import *

In [ ]:
path_dir= "/content/smearbinary/herlev/"
output_dir = "/content/herle2005Format"
datasetdiv1 = DatasetDivision(path_dir,output_dir)
datasetdiv1.printnow("lets go")
datasetdiv1.divide_dataset(path_dir, output_dir)

Instance of the class created
Just testing that the method calling is workinglets go
Dataset Division finished.
Dataset Division finished.


In [ ]:
#data augmentation
#directories
target_directory = "/content/herle2005Format/train"
#create an instance of the class
datasetda = DataAugmentation_Extension()
datasetda.extend_dataset(target_directory)

Instance of DataAugmentation_Extension class created
HEY
Instance of the DataAugmentation class created
Directory exists
['/content/herle2005Format/train/Normal/209566047-209566095-001.BMP', '/content/herle2005Format/train/Normal/209522641-209522674-001.BMP', '/content/herle2005Format/train/Normal/153958547-153958572-006.BMP', '/content/herle2005Format/train/Normal/209565864-209565911-001.BMP', '/content/herle2005Format/train/Normal/157183828-157183877-001.BMP', '/content/herle2005Format/train/Normal/209047526-209047762-001.BMP', '/content/herle2005Format/train/Normal/153956444-153956458-001.BMP', '/content/herle2005Format/train/Normal/157185781-157185814-001.BMP', '/content/herle2005Format/train/Normal/157185730-157185758-002.BMP', '/content/herle2005Format/train/Normal/157185433-157185479-003.BMP', '/content/herle2005Format/train/Normal/209565864-209565950-001.BMP', '/content/herle2005Format/train/Normal/157224297-157224336-001.BMP', '/content/herle2005Format/train/Normal/157181671-1

**Feature Combination**

In [ ]:
import numpy as np
from keras.layers import Dense, BatchNormalization, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


"""LOAD DATA"""
# path to folder with data
path = "/content/herle2005Format/"
# expected data structure inside folder: train, test, val. in each folder: one folder for each class,
# comprising its respective slide images.


# specify image data generator with data augmentation (train_datagen) resp. without (no_DA_IDG)
train_datagen = ImageDataGenerator(featurewise_center=False,
                                    rotation_range = 5, fill_mode="nearest",
                                    zoom_range=[1/1.0, 1/1.0], width_shift_range=0.0, height_shift_range=0.0, # occasionally out of range
                                    horizontal_flip = True, vertical_flip=True,
                                    brightness_range=[0.5, 1.3], channel_shift_range=20)

no_DA_IDG = ImageDataGenerator()

# in training set: use data augmentation image data generator, for validation and test: no data augmentation.
training_set = train_datagen.flow_from_directory(path+"train/",
                                                target_size=(224, 224), # typical imagenet dimensions
                                                color_mode='rgb',
                                                batch_size=32,
                                                class_mode='categorical', shuffle=True)

training_set_V2 =  no_DA_IDG.flow_from_directory(path+"train/", ### TO USE FOR FEATURE EXTRACTION
                                                target_size=(224, 224), # typical imagenet dimensions
                                                color_mode='rgb',
                                                batch_size=1,
                                                class_mode='categorical', shuffle=False)

validation_set = no_DA_IDG.flow_from_directory(path+"val/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                class_mode='categorical',
                                                shuffle=True)

validation_set_V2 = no_DA_IDG.flow_from_directory(path+"val/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=1,
                                                class_mode='categorical',
                                                shuffle=False)

test_set_V2 = no_DA_IDG.flow_from_directory(path+"test/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=1,
                                                class_mode='categorical',
                                                shuffle=False)

###############################################################################################
"""CREATE MODELS"""
from keras.models import Sequential, load_model
model_vgg = load_model('model_vgg16_herlev_binary.h5') 
model_xcep = load_model('model_xcep_herlev_binary.h5')
model_vgg19 = load_model('model_vgg19_herlev_binary.h5')
model_res50 = load_model('model_R50_T2_herlev_binary.h5')

"""LOADING WEIGHTS FROM PREVIOUSLY TRAINED MODELS"""
model_xcep.load_weights("K_xcep_T2_herlev_binary.h5")
model_vgg.load_weights("K_VGG16_T2_herlev_binary.h5")
model_res50.load_weights("K_R50_T2_herlev_binary.h5")
model_vgg19.load_weights("K_VGG19_T2_herlev_binary.h5")



"""MODEL COMBINATION"""
# creating the model extracting features from the last layer before the softmax layer.
vgg_extractor = Model(inputs=model_vgg.input, outputs=model_vgg.get_layer("dense_1024").output)
xcep_extractor = Model(inputs=model_xcep.input, outputs=model_xcep.get_layer("dense_1024").output)
vgg19_extractor = Model(inputs=model_vgg19.input, outputs=model_vgg19.get_layer("dense_1024").output)
r50_extractor = Model(inputs=model_res50.input, outputs=model_res50.get_layer("dense_1024").output)


y_train = to_categorical(training_set_V2.classes)
X_train_m1 = vgg_extractor.predict_generator(training_set_V2, steps = training_set_V2.n)
X_train_m2 = xcep_extractor.predict_generator(training_set_V2, steps = training_set_V2.n)
X_train_m3 = vgg19_extractor.predict_generator(training_set_V2, steps = training_set_V2.n)
X_train_m4 = r50_extractor.predict_generator(training_set_V2, steps = training_set_V2.n)
X_train = np.concatenate([X_train_m1, X_train_m2, X_train_m3, X_train_m4], axis=1)

y_val = to_categorical(validation_set_V2.classes)
X_val_m1 = vgg_extractor.predict_generator(validation_set_V2, steps = validation_set_V2.n)
X_val_m2 = xcep_extractor.predict_generator(validation_set_V2, steps = validation_set_V2.n)
X_val_m3 = vgg19_extractor.predict_generator(validation_set_V2, steps = validation_set_V2.n)
X_val_m4 = r50_extractor.predict_generator(validation_set_V2, steps = validation_set_V2.n)
X_val = np.concatenate([X_val_m1, X_val_m2, X_val_m3, X_val_m4], axis=1)

y_test = to_categorical(test_set_V2.classes)
X_test_m1 = vgg_extractor.predict_generator(test_set_V2, steps = test_set_V2.n)
X_test_m2 = xcep_extractor.predict_generator(test_set_V2, steps = test_set_V2.n)
X_test_m3 = vgg19_extractor.predict_generator(test_set_V2, steps = test_set_V2.n)
X_test_m4 = r50_extractor.predict_generator(test_set_V2, steps = test_set_V2.n)
X_test = np.concatenate([X_test_m1, X_test_m2, X_test_m3, X_test_m4], axis=1)


"""TRAIN & TEST FEATURE EXTRACTION MODEL"""
# the feature arrays are read into a sequential model directly connecting them to the softmax layer, with
# some dropout and batch normalization in between.

np.random.seed(668)

opt = Adam(learning_rate=1e-3)
model = Sequential()
model.add(Dropout(0.5, input_shape=(4096,)))
model.add(BatchNormalization())
model.add(Dense(2, activation="softmax"))
model1=model
model1.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])


epochs = 200
history1=model1.fit(X_train, y_train, batch_size=32, epochs=epochs, verbose=2, shuffle=True, validation_data=(X_val, y_val))

"""EVALUATE MODEL ON TEST DATA"""
y_test = np.argmax(y_test, axis=1)
pred = np.argmax(model1.predict(X_test), axis=1)
model1.save('model_binary_vgg16_vgg19_Res50_xception_herlev7class.h5')
print(accuracy_score(y_test, pred))
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

Found 8235 images belonging to 2 classes.
Found 8235 images belonging to 2 classes.
Found 184 images belonging to 2 classes.
Found 184 images belonging to 2 classes.
Found 184 images belonging to 2 classes.
Instructions for updating:
Please use Model.predict, which supports generators.
Epoch 1/200
258/258 - 1s - loss: 0.1519 - acc: 0.9477 - val_loss: 0.0500 - val_acc: 0.9837
Epoch 2/200
258/258 - 1s - loss: 0.1078 - acc: 0.9611 - val_loss: 0.0486 - val_acc: 0.9891
Epoch 3/200
258/258 - 1s - loss: 0.1074 - acc: 0.9620 - val_loss: 0.0695 - val_acc: 0.9783
Epoch 4/200
258/258 - 1s - loss: 0.1037 - acc: 0.9678 - val_loss: 0.0472 - val_acc: 0.9837
Epoch 5/200
258/258 - 1s - loss: 0.0965 - acc: 0.9679 - val_loss: 0.0570 - val_acc: 0.9837
Epoch 6/200
258/258 - 1s - loss: 0.1060 - acc: 0.9628 - val_loss: 0.0449 - val_acc: 0.9837
Epoch 7/200
258/258 - 1s - loss: 0.0916 - acc: 0.9702 - val_loss: 0.0586 - val_acc: 0.9783
Epoch 8/200
258/258 - 1s - loss: 0.0902 - acc: 0.9728 - val_loss: 0.0398 - v